# Python Intro - Final Exercise


In [20]:
import pyodbc
import pandas as pd
import numpy as np

In [21]:
### In windows, Using a ODBC DNS (predefined connection name)
conn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=RAS-SHIVUK-SHAY\SQLEXPRESS;"
                      "Database=COLLEGE;"
                      "Trusted_Connection=yes;")

In [22]:
### Get the students table
df_students = pd.read_sql("SELECT * FROM Students;",conn)
df_teachers= pd.read_sql("SELECT * FROM Teachers;",conn)
df_departments= pd.read_sql("SELECT * FROM Departments;",conn)
df_courses= pd.read_sql("SELECT * FROM Courses;",conn)
df_classrooms= pd.read_sql("SELECT * FROM Classrooms;",conn)

# Questions
 
 
## Q1. Count the number of students on each department

In [23]:
df_departments = df_departments.rename(columns={'DepartmentId': 'DepartmentID'})
 
Students_Course = pd.merge(df_students, df_classrooms, on ='StudentId')

Students_Course_Department = pd.merge(Students_Course, df_courses,on='CourseId')
 
Student_Dep = pd.merge(Students_Course_Department, df_departments, on='DepartmentID')
 
Students_Dep=Student_Dep.groupby(['DepartmentID','DepartmentName']).agg("StudentId").nunique()

Students_Dep

DepartmentID  DepartmentName
1.0           English            84
2.0           Science           200
3.0           Arts              137
4.0           Sport              81
Name: StudentId, dtype: int64

In [24]:
## Q2. How many students have each course of the English department and the total number of students in the department?

In [25]:
depid = df_departments.loc[df_departments["DepartmentName"] == "English", "DepartmentID"].iloc[0]

total_in_english = 0

courses = df_courses.loc[df_courses["DepartmentID"] == depid, ["CourseId", "CourseName"]]
num_courses = courses.count().iloc[0]

students_count = []

for num in range(int(num_courses)):

    course = df_courses.iloc[[num]]
    course_id = int(course["CourseId"])

    num_of_students_in_course = int(df_classrooms.loc[df_classrooms["CourseId"] == course_id, "CourseId"].count())

    students_count.append(num_of_students_in_course)

    total_in_english += num_of_students_in_course # same as: total = total + num_of...

courses["students"] = students_count

del courses["CourseId"]

courses.loc[num_courses] = ["total", total_in_english]

courses

,CourseName,students
0,English Begginers,32
1,Advanced English,24
2,Proffesional English,38
3,total,94


## Q3. How many small (<22 students) and large (22+ students) classrooms are needed for the Science department?

In [26]:
n_by_course = df_classrooms.groupby("CourseId").agg({"StudentId": np.count_nonzero})

n_by_course["students_num"] = n_by_course["StudentId"]
students_courses = n_by_course.groupby(["StudentId"]).agg({"students_num": np.sum})
def class_size (row):
   if row ['students_num'] > 22 :
      return 'small'
   else :
      return 'big'

students_courses["class_size"] = students_courses.apply(lambda row: class_size(row), axis=1)

count_big = students_courses.where(students_courses["class_size"] == 'big').dropna().groupby("StudentId").agg({"class_size": np.count_nonzero})
count_small = students_courses.where(students_courses["class_size"] == 'small').dropna().groupby("StudentId").agg({"class_size": np.count_nonzero})
class_type = pd.DataFrame([count_big.class_size, count_small.class_size], index = ["big_class", "small_class"])
class_type

StudentId,10.0,12.0,14.0,16.0,18.0,24.0,26.0,30.0,32.0,34.0,36.0,38.0,40.0
big_class,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
small_class,NaN,1.0,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


## Q4. A feminist student claims that there are more male than female in the College. Justify if the argument is correct

In [27]:
df_students.groupby(["Gender"] )['StudentId'].count()

Gender
F    165
M    115
Name: StudentId, dtype: int64

## Q5. For which courses the percentage of male/female students is over 70%?

In [28]:
df_students = df_students.dropna()

df_classrooms = df_classrooms.dropna()

def find_gender(row):
    
    return df_students[df_students["StudentId"] == row["StudentId"]]["Gender"].iloc[0]
        
classrooms_gender = df_classrooms
classrooms_gender["Gender"] = classrooms_gender.apply(lambda row: find_gender(row), axis=1)
count_F = classrooms_gender.where(classrooms_gender["Gender"] == 'F').dropna().groupby("CourseId").agg({"Gender": np.count_nonzero})
count_F = count_F.rename(columns={'Gender':'count_F'})
count_M = classrooms_gender.where(classrooms_gender["Gender"] == 'M').dropna().groupby("CourseId").agg({"Gender": np.count_nonzero})
count_M = count_M.rename(columns={'Gender':'count_M'})
count_f_m = pd.merge(count_F, count_M, how='outer', left_index=True, right_index=True)
count_f_m = count_f_m.astype(int)
count_f_m
count_f_m["Gender"] = "F"
count_f_m["studentPrecent"] = (count_f_m["count_F"]/(count_f_m["count_M"]+count_f_m["count_F"]))*100
result = count_f_m.where(count_f_m["studentPrecent"] > 70).dropna()
clear_result = result.drop(columns = ["count_F", "count_M"])
ResultQ3 = pd.merge(df_courses, clear_result, how='inner', left_index=True, right_index=True)
ResultQ3 = ResultQ3.drop(columns = ["DepartmentID", "TeacherId"])
ResultQ3

,CourseId,CourseName,Gender,studentPrecent
22.0,23.0,Musicology,F,70.833333
29.0,30.0,Swimming,F,72.222222


## Q6. For each department, how many students passed with a grades over 80?

In [29]:
df_departments = df_departments.rename(columns={'DepartmentId': 'DepartmentID'})
df_merged = df_classrooms.merge( df_courses, left_on="CourseId", right_on="CourseId")
df_merged = df_merged.merge( df_students, left_on="StudentId", right_on="StudentId")
df_merged = df_merged.merge( df_departments, left_on="DepartmentID", right_on="DepartmentID")
df_merged = df_merged.merge( df_teachers, left_on="TeacherId", right_on='TeacherId', suffixes=('', '_teacher'))
#df_merged
#df_merged = df_merged.replace(r'\W+', '', regex=True)
student_average = df_merged.groupby(["DepartmentName","StudentId"]).agg(grade=('degree','mean'))
summary = student_average.groupby(["DepartmentName"]).agg(total=('grade','count'))
summary['above_80'] = student_average[student_average['grade']>80].groupby(['DepartmentName']).count()
summary['perc'] = summary['above_80'] / summary['total'] * 100
summary

,total,above_80,perc
DepartmentName,,,
Arts,137,50,36.496350
English,84,37,44.047619
Science,200,86,43.000000
Sport,81,36,44.444444


## Q7. For each department, how many students passed with a grades under 60?

In [30]:
student_average = df_merged.groupby(["DepartmentName","StudentId"]).agg(grade=('degree','mean'))
summary = student_average.groupby(["DepartmentName"]).agg(total=('grade','count'))
summary['below_60'] = student_average[student_average['grade']<60].groupby(['DepartmentName']).count()
summary['perc'] = summary['below_60'] / summary['total'] * 100
summary

,total,below_60,perc
DepartmentName,,,
Arts,137,3,2.189781
English,84,1,1.190476
Science,200,2,1.000000
Sport,81,1,1.234568


## Q8. Rate the teachers by their average student's grades (in descending order).

In [31]:
#print(df_merged['FirstName_teacher'].unique())
teacher_student_average = df_merged.groupby(['FirstName_teacher', 'LastName_teacher']).agg(
                        avg_grade=('degree','mean')).sort_values(by='avg_grade', ascending=False)
teacher_student_average

,,avg_grade
FirstName_teacher,LastName_teacher,
Darick,Hess,81.432703
Jordyn,Baron,80.729494
Heather,Metcalf,80.440242
Zachary,Healy,79.365867
Kaylea,Monzingo,79.365537
Daniel,Bland,78.908281
Keegan,Price,78.814631
Conor,Mckittrick,78.699492
Maxwell,Romkee,78.673509


## Q9. Create a dataframe showing the courses, departments they are associated with, the teacher in each course, and the number of students enrolled in the course (for each course, department and teacher show the names).

In [32]:
course_list = df_merged.groupby(['DepartmentName', 'CourseName', 'FirstName_teacher', 'LastName_teacher']).agg(student_count=('StudentId','nunique'))
course_list

student_count
DepartmentName CourseName           FirstName_teacher LastName_teacher               
Arts           History of Arts       Caroline         Knuesel                      32
               Literature            Daniel           Bland                        39
               Musicology            Madison          Batan                        30
               Painting              Katerina         Young                        34
               Sculpture             Robert           Aivazian                     23
               Theater               Ernest           Stoyanov                     26
English        Advanced English      Conor            Mckittrick                   24
               English Begginers     Conor            Mckittrick                   32
               Proffesional English  Conor            Mckittrick                   38
Science        Algebra               Alexander        Watts                        12
               Biology               Jordyn           Baron                        36
               Chemistry             Jeffrey          Batzel                       23
               Computer Science      Kaylea           Monzingo                     38
               Data Science          Mackenzie        Olsen                        30
               Genetics              Terran           Beaty                        37
               Geometry              Alexander        Watts                        13
               Java                  Darick           Hess                         16
               Matematics Advanced   Alexander        Watts                        10
               Mathematics 101       Amanda           Dworkin                      18
               Neurobiology          Terran           Beaty                        24
               Physics               Alexander        Watts                        12
               Python                Terran           Beaty                        23
               R                     Tanner           Fitzsimons                   33
               SQL                   Zachary          Healy                        24
               Statistics            Heather          Metcalf                      24
Sport          Gymnastics            Maxwell          Romkee                       22
               Swimming              Mikayla          Russell                      18
               Tenis                 Keegan           Price                        18
               Volleyball            Kaitlin          Delude                       36

In [33]:
## Q10. Create a dataframe showing the students, the number of courses they take, 
##the average of the grades per class, and their overall average (for each student show the student name).

In [34]:
S_CL = pd.merge(df_students, df_classrooms, on='StudentId')

S_CL_C =  pd.merge(S_CL, df_courses, on='CourseId')

S_CL_C_D =  pd.merge(S_CL_C, df_departments, on='DepartmentID')

student_courses = S_CL_C_D.groupby(['StudentId','FirstName','LastName']).agg(courses=('CourseId','nunique'))

student_courses['General'] = S_CL_C_D.groupby(['StudentId','FirstName','LastName']).agg(General=('degree','mean'))

for department_name in S_CL_C_D['DepartmentName'].unique():
    student_courses[department_name] = S_CL_C_D[S_CL_C_D['DepartmentName']==department_name].groupby(['StudentId','FirstName','LastName']).agg(department_name=( 'degree', 'mean' ))
student_courses

,,,courses,General,Science,Arts,Sport,English
StudentId,FirstName,LastName,,,,,,
1.0,Alexandria,Heller,4,81.337515,79.557508,86.677535,NaN,NaN
2.0,Cassie,Liska,5,79.137364,78.867115,79.542738,NaN,NaN
3.0,Hayley,Frank,3,78.793700,76.204427,NaN,NaN,83.972245
4.0,Maxwell,Kotch,3,71.481003,74.157066,69.356144,NaN,70.929801
5.0,Dylan,Boehm,3,76.718650,100.000000,64.494845,65.661106,NaN
...,...,...,...,...,...,...,...,...
275.0,Elise,Ohler,4,82.028830,80.897496,94.702880,NaN,71.617451
276.0,Dylan,Bannister,3,76.784735,76.784735,NaN,NaN,NaN
277.0,Bryan,Medina,6,80.482335,81.298837,78.849330,NaN,NaN
